In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/auto-sales-data/Auto Sales data.csv


In [2]:
# Importing the necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [3]:
# Creating a Decision Tree Regressor
dt_model = DecisionTreeRegressor(random_state=42)
dt_model

DecisionTreeRegressor(random_state=42)

In [4]:
df = pd.read_csv('/kaggle/input/auto-sales-data/Auto Sales data.csv')
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,DAYS_SINCE_LASTORDER,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,CITY,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,24/02/2018,828,Shipped,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,897 Long Airport Avenue,NYC,10022,USA,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,07/05/2018,757,Shipped,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,59 rue de l'Abbaye,Reims,51100,France,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,01/07/2018,703,Shipped,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,75508,France,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,25/08/2018,649,Shipped,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,78934 Hillside Dr.,Pasadena,90003,USA,Young,Julie,Medium
4,10168,36,96.66,1,3479.76,28/10/2018,586,Shipped,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,9408 Furth Circle,Burlingame,94217,USA,Hirano,Juri,Medium


In [5]:
df = df.drop(['ORDERDATE','POSTALCODE','CONTACTLASTNAME', 'CONTACTFIRSTNAME','ORDERNUMBER','PHONE', 'ADDRESSLINE1','POSTALCODE'],axis=1)

#df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'],yearfirst = True ) # changing the date time format to a suitable format

df.head()

,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,DAYS_SINCE_LASTORDER,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,COUNTRY,DEALSIZE
0,30,95.70,2,2871.00,828,Shipped,Motorcycles,95,S10_1678,Land of Toys Inc.,NYC,USA,Small
1,34,81.35,5,2765.90,757,Shipped,Motorcycles,95,S10_1678,Reims Collectables,Reims,France,Small
2,41,94.74,2,3884.34,703,Shipped,Motorcycles,95,S10_1678,Lyon Souveniers,Paris,France,Medium
3,45,83.26,6,3746.70,649,Shipped,Motorcycles,95,S10_1678,Toys4GrownUps.com,Pasadena,USA,Medium
4,36,96.66,1,3479.76,586,Shipped,Motorcycles,95,S10_1678,Technics Stores Inc.,Burlingame,USA,Medium


In [6]:
df.nunique()

QUANTITYORDERED           58
PRICEEACH               1843
ORDERLINENUMBER           18
SALES                   2690
DAYS_SINCE_LASTORDER    1832
STATUS                     6
PRODUCTLINE                7
MSRP                      80
PRODUCTCODE              109
CUSTOMERNAME              89
CITY                      71
COUNTRY                   19
DEALSIZE                   3
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747 entries, 0 to 2746
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   QUANTITYORDERED       2747 non-null   int64  
 1   PRICEEACH             2747 non-null   float64
 2   ORDERLINENUMBER       2747 non-null   int64  
 3   SALES                 2747 non-null   float64
 4   DAYS_SINCE_LASTORDER  2747 non-null   int64  
 5   STATUS                2747 non-null   object 
 6   PRODUCTLINE           2747 non-null   object 
 7   MSRP                  2747 non-null   int64  
 8   PRODUCTCODE           2747 non-null   object 
 9   CUSTOMERNAME          2747 non-null   object 
 10  CITY                  2747 non-null   object 
 11  COUNTRY               2747 non-null   object 
 12  DEALSIZE              2747 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 279.1+ KB


In [8]:
label_encoder = LabelEncoder()

df['STATUS'] = label_encoder.fit_transform(df['STATUS'])
df['PRODUCTLINE'] = label_encoder.fit_transform(df['PRODUCTLINE'])
df['PRODUCTCODE'] = label_encoder.fit_transform(df['PRODUCTCODE'])
df['CUSTOMERNAME'] = label_encoder.fit_transform(df['CUSTOMERNAME'])
df['CITY'] = label_encoder.fit_transform(df['CITY'])
df['COUNTRY'] = label_encoder.fit_transform(df['COUNTRY'])
df['DEALSIZE'] = label_encoder.fit_transform(df['DEALSIZE'])

df.head()

,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,DAYS_SINCE_LASTORDER,STATUS,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,CITY,COUNTRY,DEALSIZE
0,30,95.70,2,2871.00,828,5,1,95,0,44,41,18,2
1,34,81.35,5,2765.90,757,5,1,95,0,63,55,6,2
2,41,94.74,2,3884.34,703,5,1,95,0,45,51,6,1
3,45,83.26,6,3746.70,649,5,1,95,0,82,52,18,1
4,36,96.66,1,3479.76,586,5,1,95,0,76,12,18,1


In [9]:
# 'SALES' is the target variable
X = df.drop('SALES', axis=1)  # Features
y = df['SALES']  # Target variable

In [10]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Training the model
dt_model.fit(X_train, y_train)# enable_categorical = True)

DecisionTreeRegressor(random_state=42)

In [12]:
# Making predictions on the test set
y_pred = dt_model.predict(X_test)
y_pred

array([2487.45, 1958.84, 2538.8 , 2586.88, 1764.75, 3988.6 , 2152.02,
       7110.91, 3222.  , 6075.3 , 3091.19, 2204.6 , 1266.1 , 3385.9 ,
       3131.2 , 5481.45, 6761.6 , 2980.6 , 6862.94, 2455.02, 6996.42,
       2530.01, 2441.04, 1979.6 , 1513.54, 1246.44, 3155.49, 6930.74,
       2790.45, 4816.08, 3184.8 , 5074.39, 5512.32, 3957.05, 4808.38,
       3611.16, 5195.85, 4509.12, 2230.41, 1917.84, 1875.2 , 1277.64,
       2455.02, 4459.62, 2862.72, 2299.  , 3881.25, 1987.02, 7023.9 ,
       3179.52, 3329.09, 1843.52, 2144.6 , 1352.4 , 6311.2 , 2351.36,
       6761.6 , 6295.95, 1605.  , 1707.14, 3666.96, 6539.04, 3559.4 ,
       4103.46, 6231.91, 7483.98, 1527.8 , 2999.97, 3974.94, 2824.8 ,
       1624.25, 6763.05, 2203.11, 1767.43, 6668.24, 6083.  , 3581.4 ,
       4431.84, 1930.5 , 5700.09, 1696.82, 2113.37, 1246.44, 5096.91,
       2814.28, 2754.7 , 3657.69, 2046.  , 3177.3 , 5723.78, 3431.25,
       2294.18, 5760.93, 2038.87, 5884.65, 3239.62, 2349.9 , 1987.02,
       5096.91, 4219

In [13]:
# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Absolute Error: 118.9059272727273
Mean Squared Error: 42127.24253236363
R-squared: 0.9861802715821307


**Hyperparameter Tuning**

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
# Defining the hyperparameters and their possible values
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [16]:
# Creating a GridSearchCV object
grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)

grid_search

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_absolute_error')

In [17]:
# Performing the grid search on the training data
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_absolute_error')

In [18]:
# Getting the best hyperparameters from the grid search
best_params = grid_search.best_params_

best_params

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}

In [19]:
# Creating a Decision Tree model with the best hyperparameters
best_dt_model = DecisionTreeRegressor(**best_params)

best_dt_model

DecisionTreeRegressor(min_samples_split=5)

In [20]:
# Training the model
best_dt_model.fit(X_train, y_train)

DecisionTreeRegressor(min_samples_split=5)

In [21]:
# Make predictions on the test set
y_pred = best_dt_model.predict(X_test)

y_pred

array([ 2487.45      ,  1976.6       ,  2547.49      ,  2568.10666667,
        1733.49333333,  3975.46333333,  2179.2875    ,  7071.01333333,
        3196.29      ,  6066.6225    ,  3030.25      ,  2212.53      ,
        1313.19      ,  3386.365     ,  3131.795     ,  5404.34333333,
        6865.8       ,  2932.05666667,  6884.5       ,  2403.19      ,
        6954.0875    ,  2528.365     ,  2439.9225    ,  1972.97333333,
        1527.15      ,  1251.19666667,  3146.8225    ,  6865.8       ,
        2818.66      ,  4940.28      ,  3196.29      ,  5248.        ,
        5478.515     ,  3925.41666667,  4734.5125    ,  3611.16      ,
        5179.7525    ,  4509.12      ,  2212.53      ,  1917.89666667,
        1889.91      ,  1251.19666667,  2403.19      ,  4489.87      ,
        2812.64666667,  2325.8025    ,  3908.285     ,  2011.1175    ,
        7071.01333333,  3179.52      ,  3293.3925    ,  1860.5325    ,
        2143.3       ,  1400.425     ,  6294.1325    ,  2403.19      ,
      

In [22]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Mean Absolute Error: 113.06367575757577
Mean Squared Error: 33371.039809808084
R-squared: 0.9890527202952066


In [23]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [24]:
# Defining hyperparameter grid for tuning
param_dist = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


In [25]:
# Using Randomized Search to find the best hyperparameters
random_search = RandomizedSearchCV(estimator=dt_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
random_search

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10]},
                   scoring='neg_mean_squared_error')

In [26]:
# Fitting the grid search to the data
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10]},
                   scoring='neg_mean_squared_error')

In [27]:
# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

Best Hyperparameters: {'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': None}


In [28]:
# Get the best model from the random search
best_dt_model = random_search.best_estimator_
best_dt_model

DecisionTreeRegressor(min_samples_split=5, random_state=42)

In [29]:
# Training the model
best_dt_model.fit(X_train, y_train)

DecisionTreeRegressor(min_samples_split=5, random_state=42)

In [30]:
# Make predictions on the test set
y_pred = best_dt_model.predict(X_test)

#y_pred

In [31]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Hyperparameters: {best_params}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Mean Absolute Error: 111.85344242424243
Mean Squared Error: 32666.610215257577
R-squared: 0.9892838065258974


**Before Hyperparameter Tuning**

Mean Absolute Error: 118.9059272727273

Mean Squared Error: 42127.24253236363

R-squared: 0.9861802715821307



**After Hyperparameter Tuning using GridSearchCV**

Mean Absolute Error: 115.85517575757576

Mean Squared Error: 34328.15386646971

R-squared: 0.9887387416074769



**After Hyperparameter Tuning using RandomizedSearchCV**

Mean Absolute Error: 120.25975282940558

Mean Squared Error: 37127.57185141332

R-squared: 0.9878204000794193


GridSearchCV yields better results.